In [73]:
import pandas as pd
import numpy as np
import math
from math import isnan
from pulp import *
from collections import Counter
from more_itertools import unique_everseen


In [115]:
sales=pd.read_csv("sales_lift_input.csv",header=None) #input file
sales

,0,1,2,3,4,5,6,7,8
0,Shelf,Unilever,Unilever,Unilever,Godrej,Godrej,Godrej,Dabur,Dabur
1,NaN,1,2,3,4,5,6,7,8
2,1,950,818,650,848,630,648,842,842
3,2,691,849,615,700,653,598,563,563
4,3,427,413,349,347,407,237,465,465
5,4,345,153,282,301,477,432,313,313
6,5,464,470,126,392,534,312,326,326
7,6,281,144,283,200,168,107,148,148
8,7,238,500,291,434,465,488,544,544
9,8,138,86,119,149,92,136,119,119


In [116]:
lift=sales.iloc[2:,1:]
lift

,1,2,3,4,5,6,7,8
2,950,818,650,848,630,648,842,842
3,691,849,615,700,653,598,563,563
4,427,413,349,347,407,237,465,465
5,345,153,282,301,477,432,313,313
6,464,470,126,392,534,312,326,326
7,281,144,283,200,168,107,148,148
8,238,500,291,434,465,488,544,544
9,138,86,119,149,92,136,119,119
10,127,124,141,54,130,140,75,75
11,70,141,78,106,69,51,58,58


In [117]:
lift=np.array(lift)
lift

array([['950', '818', '650', '848', '630', '648', '842', '842'],
       ['691', '849', '615', '700', '653', '598', '563', '563'],
       ['427', '413', '349', '347', '407', '237', '465', '465'],
       ['345', '153', '282', '301', '477', '432', '313', '313'],
       ['464', '470', '126', '392', '534', '312', '326', '326'],
       ['281', '144', '283', '200', '168', '107', '148', '148'],
       ['238', '500', '291', '434', '465', '488', '544', '544'],
       ['138', '86', '119', '149', '92', '136', '119', '119'],
       ['127', '124', '141', '54', '130', '140', '75', '75'],
       ['70', '141', '78', '106', '69', '51', '58', '58']], dtype=object)

In [118]:
lift = lift.astype(np.int) # read the lifts from csv
lift

array([[950, 818, 650, 848, 630, 648, 842, 842],
       [691, 849, 615, 700, 653, 598, 563, 563],
       [427, 413, 349, 347, 407, 237, 465, 465],
       [345, 153, 282, 301, 477, 432, 313, 313],
       [464, 470, 126, 392, 534, 312, 326, 326],
       [281, 144, 283, 200, 168, 107, 148, 148],
       [238, 500, 291, 434, 465, 488, 544, 544],
       [138,  86, 119, 149,  92, 136, 119, 119],
       [127, 124, 141,  54, 130, 140,  75,  75],
       [ 70, 141,  78, 106,  69,  51,  58,  58]])

In [119]:
brands=sales.iloc[0:1,:]
brands

,0,1,2,3,4,5,6,7,8
0,Shelf,Unilever,Unilever,Unilever,Godrej,Godrej,Godrej,Dabur,Dabur


In [120]:
brands=np.array(brands)
brands

array([['Shelf', 'Unilever', 'Unilever', 'Unilever', 'Godrej', 'Godrej',
        'Godrej', 'Dabur', 'Dabur']], dtype=object)

In [121]:
brands=np.delete(brands,0)
brands

array(['Unilever', 'Unilever', 'Unilever', 'Godrej', 'Godrej', 'Godrej',
       'Dabur', 'Dabur'], dtype=object)

In [122]:
brands=brands.tolist()  # read the brands from csv
brands

['Unilever',
 'Unilever',
 'Unilever',
 'Godrej',
 'Godrej',
 'Godrej',
 'Dabur',
 'Dabur']

In [123]:
ff=Counter(brands)
ff

Counter({'Dabur': 2, 'Godrej': 3, 'Unilever': 3})

In [124]:
all_brands=ff.items()
all_brands

[('Dabur', 2), ('Unilever', 3), ('Godrej', 3)]

In [125]:
ff=Counter(brands)
ff

Counter({'Dabur': 2, 'Godrej': 3, 'Unilever': 3})

In [126]:

all_brands=ff.items()
all_brands

[('Dabur', 2), ('Unilever', 3), ('Godrej', 3)]

In [128]:
rack_shelf=[[1,1,2,3],[2,4,5,6],[3,7,8,9,10]]
rack_shelf



[[1, 1, 2, 3], [2, 4, 5, 6], [3, 7, 8, 9, 10]]

In [129]:
prob=LpProblem("SO",LpMaximize)
prob

SO:
MAXIMIZE
None
VARIABLES

In [130]:
dec_var=LpVariable.matrix("dec_var",(range(len(lift)),range(len(lift[0]))),0,1,LpBinary)
dec_var

[[dec_var_0_0,
  dec_var_0_1,
  dec_var_0_2,
  dec_var_0_3,
  dec_var_0_4,
  dec_var_0_5,
  dec_var_0_6,
  dec_var_0_7],
 [dec_var_1_0,
  dec_var_1_1,
  dec_var_1_2,
  dec_var_1_3,
  dec_var_1_4,
  dec_var_1_5,
  dec_var_1_6,
  dec_var_1_7],
 [dec_var_2_0,
  dec_var_2_1,
  dec_var_2_2,
  dec_var_2_3,
  dec_var_2_4,
  dec_var_2_5,
  dec_var_2_6,
  dec_var_2_7],
 [dec_var_3_0,
  dec_var_3_1,
  dec_var_3_2,
  dec_var_3_3,
  dec_var_3_4,
  dec_var_3_5,
  dec_var_3_6,
  dec_var_3_7],
 [dec_var_4_0,
  dec_var_4_1,
  dec_var_4_2,
  dec_var_4_3,
  dec_var_4_4,
  dec_var_4_5,
  dec_var_4_6,
  dec_var_4_7],
 [dec_var_5_0,
  dec_var_5_1,
  dec_var_5_2,
  dec_var_5_3,
  dec_var_5_4,
  dec_var_5_5,
  dec_var_5_6,
  dec_var_5_7],
 [dec_var_6_0,
  dec_var_6_1,
  dec_var_6_2,
  dec_var_6_3,
  dec_var_6_4,
  dec_var_6_5,
  dec_var_6_6,
  dec_var_6_7],
 [dec_var_7_0,
  dec_var_7_1,
  dec_var_7_2,
  dec_var_7_3,
  dec_var_7_4,
  dec_var_7_5,
  dec_var_7_6,
  dec_var_7_7],
 [dec_var_8_0,
  dec_var_8_1,
  

In [65]:
prodt_matrix=[dec_var[i][j]*lift[i][j] for i in range(len(lift))
for j in range(len(lift[0]))]


In [66]:
prodt_matrix

[LpAffineExpression([(dec_var_0_0, 950)]),
 LpAffineExpression([(dec_var_0_1, 818)]),
 LpAffineExpression([(dec_var_0_2, 650)]),
 LpAffineExpression([(dec_var_0_3, 848)]),
 LpAffineExpression([(dec_var_0_4, 630)]),
 LpAffineExpression([(dec_var_0_5, 648)]),
 LpAffineExpression([(dec_var_0_6, 842)]),
 LpAffineExpression([(dec_var_0_7, 842)]),
 LpAffineExpression([(dec_var_1_0, 691)]),
 LpAffineExpression([(dec_var_1_1, 849)]),
 LpAffineExpression([(dec_var_1_2, 615)]),
 LpAffineExpression([(dec_var_1_3, 700)]),
 LpAffineExpression([(dec_var_1_4, 653)]),
 LpAffineExpression([(dec_var_1_5, 598)]),
 LpAffineExpression([(dec_var_1_6, 563)]),
 LpAffineExpression([(dec_var_1_7, 563)]),
 LpAffineExpression([(dec_var_2_0, 427)]),
 LpAffineExpression([(dec_var_2_1, 413)]),
 LpAffineExpression([(dec_var_2_2, 349)]),
 LpAffineExpression([(dec_var_2_3, 347)]),
 LpAffineExpression([(dec_var_2_4, 407)]),
 LpAffineExpression([(dec_var_2_5, 237)]),
 LpAffineExpression([(dec_var_2_6, 465)]),
 LpAffineEx

In [68]:
prob+=lpSum(prodt_matrix)

In [69]:
prob

SO:
MAXIMIZE
950*dec_var_0_0 + 818*dec_var_0_1 + 650*dec_var_0_2 + 848*dec_var_0_3 + 630*dec_var_0_4 + 648*dec_var_0_5 + 842*dec_var_0_6 + 842*dec_var_0_7 + 691*dec_var_1_0 + 849*dec_var_1_1 + 615*dec_var_1_2 + 700*dec_var_1_3 + 653*dec_var_1_4 + 598*dec_var_1_5 + 563*dec_var_1_6 + 563*dec_var_1_7 + 427*dec_var_2_0 + 413*dec_var_2_1 + 349*dec_var_2_2 + 347*dec_var_2_3 + 407*dec_var_2_4 + 237*dec_var_2_5 + 465*dec_var_2_6 + 465*dec_var_2_7 + 345*dec_var_3_0 + 153*dec_var_3_1 + 282*dec_var_3_2 + 301*dec_var_3_3 + 477*dec_var_3_4 + 432*dec_var_3_5 + 313*dec_var_3_6 + 313*dec_var_3_7 + 464*dec_var_4_0 + 470*dec_var_4_1 + 126*dec_var_4_2 + 392*dec_var_4_3 + 534*dec_var_4_4 + 312*dec_var_4_5 + 326*dec_var_4_6 + 326*dec_var_4_7 + 281*dec_var_5_0 + 144*dec_var_5_1 + 283*dec_var_5_2 + 200*dec_var_5_3 + 168*dec_var_5_4 + 107*dec_var_5_5 + 148*dec_var_5_6 + 148*dec_var_5_7 + 238*dec_var_6_0 + 500*dec_var_6_1 + 291*dec_var_6_2 + 434*dec_var_6_3 + 465*dec_var_6_4 + 488*dec_var_6_5 + 544*dec_var_6_6

In [79]:
order=list(unique_everseen(brands))
order

['Unilever', 'Godrej', 'Dabur']

In [80]:
order_map = {}
for pos, item in enumerate(order):
    order_map[item] = pos
order_map

{'Dabur': 2, 'Godrej': 1, 'Unilever': 0}

In [81]:
brands_lift=sorted(all_brands, key=lambda x: order_map[x[0]])
brands_lift

[('Unilever', 2), ('Godrej', 3), ('Dabur', 2)]

In [85]:
for i in range(len(lift)):
    prob+=lpSum(dec_var[i])<=1
    print prob

SO:
MAXIMIZE
950*dec_var_0_0 + 818*dec_var_0_1 + 650*dec_var_0_2 + 848*dec_var_0_3 + 630*dec_var_0_4 + 648*dec_var_0_5 + 842*dec_var_0_6 + 842*dec_var_0_7 + 691*dec_var_1_0 + 849*dec_var_1_1 + 615*dec_var_1_2 + 700*dec_var_1_3 + 653*dec_var_1_4 + 598*dec_var_1_5 + 563*dec_var_1_6 + 563*dec_var_1_7 + 427*dec_var_2_0 + 413*dec_var_2_1 + 349*dec_var_2_2 + 347*dec_var_2_3 + 407*dec_var_2_4 + 237*dec_var_2_5 + 465*dec_var_2_6 + 465*dec_var_2_7 + 345*dec_var_3_0 + 153*dec_var_3_1 + 282*dec_var_3_2 + 301*dec_var_3_3 + 477*dec_var_3_4 + 432*dec_var_3_5 + 313*dec_var_3_6 + 313*dec_var_3_7 + 464*dec_var_4_0 + 470*dec_var_4_1 + 126*dec_var_4_2 + 392*dec_var_4_3 + 534*dec_var_4_4 + 312*dec_var_4_5 + 326*dec_var_4_6 + 326*dec_var_4_7 + 281*dec_var_5_0 + 144*dec_var_5_1 + 283*dec_var_5_2 + 200*dec_var_5_3 + 168*dec_var_5_4 + 107*dec_var_5_5 + 148*dec_var_5_6 + 148*dec_var_5_7 + 238*dec_var_6_0 + 500*dec_var_6_1 + 291*dec_var_6_2 + 434*dec_var_6_3 + 465*dec_var_6_4 + 488*dec_var_6_5 + 544*dec_var_6_6

In [88]:
col_con=[1,0,0,2,2,3,1,1]
col_con

[1, 0, 0, 2, 2, 3, 1, 1]

In [92]:
dec_var=np.array(dec_var)
print dec_var.shape
dec_var

(10, 8)


array([[dec_var_0_0, dec_var_0_1, dec_var_0_2, dec_var_0_3, dec_var_0_4,
        dec_var_0_5, dec_var_0_6, dec_var_0_7],
       [dec_var_1_0, dec_var_1_1, dec_var_1_2, dec_var_1_3, dec_var_1_4,
        dec_var_1_5, dec_var_1_6, dec_var_1_7],
       [dec_var_2_0, dec_var_2_1, dec_var_2_2, dec_var_2_3, dec_var_2_4,
        dec_var_2_5, dec_var_2_6, dec_var_2_7],
       [dec_var_3_0, dec_var_3_1, dec_var_3_2, dec_var_3_3, dec_var_3_4,
        dec_var_3_5, dec_var_3_6, dec_var_3_7],
       [dec_var_4_0, dec_var_4_1, dec_var_4_2, dec_var_4_3, dec_var_4_4,
        dec_var_4_5, dec_var_4_6, dec_var_4_7],
       [dec_var_5_0, dec_var_5_1, dec_var_5_2, dec_var_5_3, dec_var_5_4,
        dec_var_5_5, dec_var_5_6, dec_var_5_7],
       [dec_var_6_0, dec_var_6_1, dec_var_6_2, dec_var_6_3, dec_var_6_4,
        dec_var_6_5, dec_var_6_6, dec_var_6_7],
       [dec_var_7_0, dec_var_7_1, dec_var_7_2, dec_var_7_3, dec_var_7_4,
        dec_var_7_5, dec_var_7_6, dec_var_7_7],
       [dec_var_8_0, dec_var_8_1

In [93]:
col_data=[]
col_data

[]

In [95]:
list(zip(*dec_var)[0])

[dec_var_0_0,
 dec_var_1_0,
 dec_var_2_0,
 dec_var_3_0,
 dec_var_4_0,
 dec_var_5_0,
 dec_var_6_0,
 dec_var_7_0,
 dec_var_8_0,
 dec_var_9_0]

In [101]:
col_data=[]
for j in range(len(brands)):
    col_data.append(list(zip(*dec_var)[j]))
    prob+=lpSum(col_data[j])<=col_con[j]


In [102]:
prob

SO:
MAXIMIZE
950*dec_var_0_0 + 818*dec_var_0_1 + 650*dec_var_0_2 + 848*dec_var_0_3 + 630*dec_var_0_4 + 648*dec_var_0_5 + 842*dec_var_0_6 + 842*dec_var_0_7 + 691*dec_var_1_0 + 849*dec_var_1_1 + 615*dec_var_1_2 + 700*dec_var_1_3 + 653*dec_var_1_4 + 598*dec_var_1_5 + 563*dec_var_1_6 + 563*dec_var_1_7 + 427*dec_var_2_0 + 413*dec_var_2_1 + 349*dec_var_2_2 + 347*dec_var_2_3 + 407*dec_var_2_4 + 237*dec_var_2_5 + 465*dec_var_2_6 + 465*dec_var_2_7 + 345*dec_var_3_0 + 153*dec_var_3_1 + 282*dec_var_3_2 + 301*dec_var_3_3 + 477*dec_var_3_4 + 432*dec_var_3_5 + 313*dec_var_3_6 + 313*dec_var_3_7 + 464*dec_var_4_0 + 470*dec_var_4_1 + 126*dec_var_4_2 + 392*dec_var_4_3 + 534*dec_var_4_4 + 312*dec_var_4_5 + 326*dec_var_4_6 + 326*dec_var_4_7 + 281*dec_var_5_0 + 144*dec_var_5_1 + 283*dec_var_5_2 + 200*dec_var_5_3 + 168*dec_var_5_4 + 107*dec_var_5_5 + 148*dec_var_5_6 + 148*dec_var_5_7 + 238*dec_var_6_0 + 500*dec_var_6_1 + 291*dec_var_6_2 + 434*dec_var_6_3 + 465*dec_var_6_4 + 488*dec_var_6_5 + 544*dec_var_6_6

In [103]:
prob.writeLP("SO.lp")

In [108]:
_

SO:
MAXIMIZE
950*dec_var_0_0 + 818*dec_var_0_1 + 650*dec_var_0_2 + 848*dec_var_0_3 + 630*dec_var_0_4 + 648*dec_var_0_5 + 842*dec_var_0_6 + 842*dec_var_0_7 + 691*dec_var_1_0 + 849*dec_var_1_1 + 615*dec_var_1_2 + 700*dec_var_1_3 + 653*dec_var_1_4 + 598*dec_var_1_5 + 563*dec_var_1_6 + 563*dec_var_1_7 + 427*dec_var_2_0 + 413*dec_var_2_1 + 349*dec_var_2_2 + 347*dec_var_2_3 + 407*dec_var_2_4 + 237*dec_var_2_5 + 465*dec_var_2_6 + 465*dec_var_2_7 + 345*dec_var_3_0 + 153*dec_var_3_1 + 282*dec_var_3_2 + 301*dec_var_3_3 + 477*dec_var_3_4 + 432*dec_var_3_5 + 313*dec_var_3_6 + 313*dec_var_3_7 + 464*dec_var_4_0 + 470*dec_var_4_1 + 126*dec_var_4_2 + 392*dec_var_4_3 + 534*dec_var_4_4 + 312*dec_var_4_5 + 326*dec_var_4_6 + 326*dec_var_4_7 + 281*dec_var_5_0 + 144*dec_var_5_1 + 283*dec_var_5_2 + 200*dec_var_5_3 + 168*dec_var_5_4 + 107*dec_var_5_5 + 148*dec_var_5_6 + 148*dec_var_5_7 + 238*dec_var_6_0 + 500*dec_var_6_1 + 291*dec_var_6_2 + 434*dec_var_6_3 + 465*dec_var_6_4 + 488*dec_var_6_5 + 544*dec_var_6_6

In [109]:
prob.solve()

1

In [110]:
print("The maximum Total lift obtained is:",value(prob.objective)) # print the output

('The maximum Total lift obtained is:', 4225.0)


In [111]:

Matrix=[[0 for X in range(len(lift[0]))] for y in range(len(lift))]
for v in prob.variables():
    Matrix[int(v.name.split("_")[2])][int(v.name.split("_")[3])]=v.varValue
    matrix=np.int_(Matrix)
print ("The decision variable matrix is:")
print(matrix)


The decision variable matrix is:
[[0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0]]


In [112]:
#Matrix=[[0 for X in range(len(lift[0]))] for y in range(len(lift))]
[0 for X in range(len(lift[0]))]

[0, 0, 0, 0, 0, 0, 0, 0]

In [113]:
[[0 for X in range(len(lift[0]))] for y in range(len(lift))]

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [114]:
for v in prob.variables():
    Matrix[int(v.name.split("_")[2])][int(v.name.split("_")[3])]=v.varValue
    matrix=np.int_(Matrix)
print ("The decision variable matrix is:")
print(matrix)

The decision variable matrix is:
[[0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0]]
